In [1]:
import qutip as qu
import jax
import numpy as np
import jax.numpy as jnp
from jax import jit
import matplotlib.pyplot as plt

In [2]:
rho = qu.rand_dm(N=2, seed=9)
rho

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.96151202+0.j        0.00221706+0.1884686j]
 [0.00221706-0.1884686j 0.03848798+0.j       ]]

In [3]:
qu.operator_to_vector(rho)

Quantum object: dims = [[[2], [2]], [1]], shape = (4, 1), type = operator-ket
Qobj data =
[[0.96151202+0.j       ]
 [0.00221706-0.1884686j]
 [0.00221706+0.1884686j]
 [0.03848798+0.j       ]]

In [4]:
N = 2  # dimension of Hilbert Space

G = np.array(
    [qu.identity(N), qu.sigmax(), qu.sigmay(), qu.sigmaz()]
) / np.sqrt(N)

In [5]:
list(G)

[array([[0.70710678+0.j, 0.        +0.j],
        [0.        +0.j, 0.70710678+0.j]]),
 array([[0.        +0.j, 0.70710678+0.j],
        [0.70710678+0.j, 0.        +0.j]]),
 array([[0.+0.j        , 0.-0.70710678j],
        [0.+0.70710678j, 0.+0.j        ]]),
 array([[ 0.70710678+0.j,  0.        +0.j],
        [ 0.        +0.j, -0.70710678+0.j]])]

In [6]:
mat_gg = np.zeros([N**2, N**2, N**2, N**2], dtype=np.complex64)
for i, gi in enumerate(list(G)):
    for j, gj in enumerate(list(G)):
        val = np.kron(np.transpose(gj), gi)
        val = val - 0.5*np.kron(np.identity(2), gj@gi)
        val = val - 0.5*np.kron(np.transpose(gj@gi), np.identity(2))
        mat_gg[i, j] =  val

0 0
0 1
0 2
0 3
1 0
1 1
1 2
1 3
2 0
2 1
2 2
2 3
3 0
3 1
3 2
3 3


In [7]:
mat_gg[1:, 1:]

array([[[[ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ],
         [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ],
         [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ],
         [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ]],

        [[ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j],
         [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ],
         [ 0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ],
         [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ]],

        [[ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ],
         [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ],
         [ 0. +0.j ,  0. +0.j ,  0. -0.j , -0.5-0.j ],
         [ 0. +0.j ,  0. +0.j , -0.5-0.j ,  0. -0.j ]]],


       [[[ 0. +0.j ,  0. -0.j ,  0. +0.j ,  0. -0.5j],
         [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ],
         [ 0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.j ],
         [ 0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ]],

        [[ 0. +0.j ,  0. -0.j ,  0. +0.j ,  0.5+0.j ],
         [ 0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ],


In [8]:
np.triu_indices(3, k=1)

(array([0, 0, 1]), array([1, 2, 2]))

In [9]:
# # I need to see how I'm going to introduce the particles. There are 5 numbers.

# # particles_vec = [a_r, a_i, b_r, b_i, c]


# def wrap_particles_to_complex(particles):
#     return np.array(
#         [
#             particles[0] + 1j * particles[1],
#             particles[2] + 1j * particles[3],
#             particles[4],
#         ],
#         dtype=np.complex64,
#     )


# def create_dij(particles, **args):
#     dij = np.zeros([N**2 - 1, N**2 - 1], dtype=np.complex64)
#     dij[np.triu_indices(3, k=1)] = [0, particles[0], particles[1]]
#     dij[np.diag_indices(N**2 - 1)] = [0, 0, particles[2]]
#     dij = dij + np.conjugate(dij.T)
#     return dij


# def dij_gg(dij, mat_gg):
#     return np.einsum("ij, ijkl -> kl", dij, mat_gg[1:, 1:])


# def make_H_re(a, b):
#     return np.array([[0, a + 1j * b], [a - 1j * b, 0]])


# def make_liouvillian(H, N=2):
#     return -1j * np.kron(H, np.identity(N)) + 1j * np.kron(np.identity(N), H)

In [10]:
delta = 0.12739334807998307
Omega = 0.5

A = np.array([[1, 0], [0, 0]])
sigmap = np.array([[0, 1], [0, 0]])
sigman = np.array([[0, 0], [1, 0]])


H0 = delta * A + Omega / 2 * (sigmap + sigman)

In [11]:
# particles_vec = [1, 2, 3, 4, 5, 6, 7]


# Hre = make_H_re(*particles_vec[0:2])

# H = H0 + Hre
# dij = create_dij(wrap_particles_to_complex(particles_vec[2:]), N=2)


# dijgg = dij_gg(dij, mat_gg)

# Ltot = make_liouvillian(H) + dijgg

In [12]:
import joblib

true_L = joblib.load("L_num.job")

In [13]:
measurement_ops = []

for i in [qu.sigmax().full(), qu.sigmay().full(), qu.sigmaz().full()]:
    measurement_ops.append(
        (0.5 * (np.identity(2) + i), 0.5 * (np.identity(2) - i))
    )

measurement_ops = jnp.array(measurement_ops)

In [14]:
d = 2
rho0 = rho.full().flatten()

Ez0 = measurement_ops[2, 0].flatten()

In [15]:
phys_model = {
    "Dim of H space": d,
    "H": H0,
    "True Lindbladian": true_L.full(),
    "Measurement operators": measurement_ops,
    "Initial state": rho0,
    "Identity matrix vectorized": jnp.eye(d).flatten("F"),
    "Identity matrix": jnp.eye(d),
    "Mat_gg": mat_gg,
}
model = {
    "Number of parameters": 8,
    "Space boundaries": jnp.array([[-1, 1] for _ in range(8)]),
    "Number of particles": 100,
    "Number of experimental repetitions": 10,
    "physical_model": phys_model,
}

In [16]:
key = jax.random.PRNGKey(seed=0)

In [17]:
def initialize_weights(model):
    N = model["Number of particles"]
    return jnp.ones(N) / N

In [18]:
def trace_vec(A, phys_model):
    return jnp.dot(A, phys_model["Identity matrix vectorized"])


@jit
def evolve(t, Liouvillian, super_rho, phys_model):
    rho = jax.scipy.linalg.expm(Liouvillian * t) @ super_rho

    return rho / trace_vec(rho, phys_model)


@jit
def compute_p(rho_vec, E_vec):
    """This should be in vectorized version.

    Args:
        rho_vec (jnp.array): Density matrix of the state
        E_vec (jnp.array): Measurement operator

    Returns:
        Real number: Probability of measure the expected outcome
    """
    p = jnp.clip(jnp.real(jnp.dot(rho_vec, E_vec)), 0.0, 1.0)

    return p


@jit
def generate_data(
    key,
    measurement_op,
    t,
    phys_model,
):
    true_L = phys_model["True Lindbladian"]
    rho0 = phys_model["Initial state"]
    rhot = evolve(t, true_L, rho0, phys_model)
    p = compute_p(rhot, measurement_op)
    key, subkey = jax.random.split(key)
    return key, jax.random.choice(
        subkey, a=jnp.array([0, 1]), p=jnp.array([1, 1 - p])
    )


def populate_one_axis(key, bnds, no_particles):
    return jax.random.uniform(
        key, minval=jnp.min(bnds), maxval=jnp.max(bnds), shape=[no_particles]
    )


def initialize_particle_locations(key, model):
    no_of_parameters = model["Number of parameters"]
    boundaries = model["Space boundaries"]
    no_of_particles = model["Number of particles"]
    subkey = jax.random.split(key, no_of_parameters + 1)
    key = subkey[1]
    subkeys = subkey[1:]
    return (
        key,
        jax.vmap(populate_one_axis, in_axes=(0, 0, None))(
            subkeys, boundaries, no_of_particles
        ).T,
    )


@jit
def create_dij(particle, model):
    d = model["Identity matrix"].shape[0]
    dij = jnp.zeros([d**2 - 1, d**2 - 1], dtype=jnp.complex64)

    dij = dij.at[0, 2].set(particle[0] + 1j * particle[1])
    dij = dij.at[1, 2].set(particle[2] + 1j * particle[3])

    dij = dij + jnp.conjugate(dij.T)
    dij = dij.at[2, 2].set(particle[4])
    return dij


@jit
def dij_gg(dij, mat_gg):
    return jnp.einsum("ij, ijkl -> kl", dij, mat_gg[1:, 1:])


@jit
def make_H_renormalization(particles_h):
    return jnp.array(
        [
            [particles_h[0], particles_h[1] + 1j * particles_h[2]],
            [particles_h[1] - 1j * particles_h[2], -particles_h[0]],
        ]
    )


@jit
def make_liouvillian(H, model):
    d = model["Identity matrix"].shape[0]
    return 1j * jnp.kron(H.T, jnp.identity(d)) - 1j * jnp.kron(
        jnp.identity(d), H
    )


@jit
def generate_liouvillian_particle(particle, model):
    H0 = model["H"]
    mat_gg = model["Mat_gg"]
    dissipator = dij_gg(create_dij(particle[0:5], model), mat_gg)
    H_renorm = make_H_renormalization(particle[5:])
    H = H0 + H_renorm
    return make_liouvillian(H, model) + dissipator


@jit
def evolve_particle(particle, t, model):
    rho0 = model["Initial state"]
    L = generate_liouvillian_particle(particle, model)
    return evolve(t, L, rho0, model)


@jit
def p_E0_particle(particle, E_0, t, model):
    return compute_p(evolve_particle(particle, t, model), E_0)


@jit
def p_data_particle(data, particle, E_0, t, model):
    p = p_E0_particle(particle, E_0, t, model)

    return jax.lax.cond(data == 0, lambda: p, lambda: 1 - p)


# vmapped version of everything.
# Given data in an array and particles in an array (particles, positions), gives
# the probability  [particles, prob associated to result]


# jax.vmap(jax.vmap(
#     p_data_particle,
#     in_axes=(0, None, None, None, None),
# ), in_axes=(None, 0, None, None, None))


@jit
def compute_likelihood_data(data, particles, E_0, t, model):
    return jnp.prod(
        jax.vmap(
            jax.vmap(
                p_data_particle,
                in_axes=(0, None, None, None, None),
            ),
            in_axes=(None, 0, None, None, None),
        )(data, particles, E_0, t, model),
        axis=1,
    )


@jit
def update_weights(lkl, weights):
    new_weights = lkl * weights
    new_weights = new_weights / jnp.sum(new_weights)
    return new_weights


@jit
def est_mean(particles_location, weights):
    return jnp.einsum("i, ij -> j", weights, particles_location)


@jit
def est_cov(particles_locations, weights):
    return jnp.einsum(
        "i, im, ik -> mk", weights, particles_locations, particles_locations
    ) - jnp.einsum(
        "i, ij, m, mk -> jk",
        weights,
        particles_locations,
        weights,
        particles_locations,
    )

In [19]:
key, particles_locations = initialize_particle_locations(key, model)
weights = initialize_weights(model)

In [27]:
cov = []

for _ in range(20):
    keys = jax.random.split(
        key, model["Number of experimental repetitions"] + 1
    )

    t = 52.9
    key = keys[0]
    _, results = jax.jit(jax.vmap(generate_data, (0, None, None, None)))(
        keys[1:], Ez0, t, model["physical_model"]
    )

    lkl = compute_likelihood_data(
        results, particles_locations, Ez0, t, model["physical_model"]
    )

    weights = update_weights(lkl, weights)
    cov.append(est_cov(particles_locations, weights))

# Finding the true values

In [43]:
def MSE(Lhat, Ltrue):
    diff = Lhat - Ltrue

    return jnp.mean(jnp.real(diff) ** 2 + jnp.imag(diff) ** 2)


@jit
def fun_to_minimize(particle, Ltrue, model):
    Lhat = generate_liouvillian_particle(particle, model)
    return MSE(Lhat, Ltrue)

# Steps

The model somehow should be independent from the algorithm, such that is easy to adapt to other situations.
The steps are:
- Generate particles
- Generate experiments. Gather data.
- Compute the likelihood of the experiments given the particles.
- Update the probability distribution
- Resample if necessary
- Check conditions to stop


In the end, we are gonna work with a set of particle locations and its associated weights. This should be the thing.